In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from phik import phik_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, learning_curve, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.calibration import calibration_curve
import optuna
from sklearn.datasets import load_breast_cancer


import warnings
warnings.filterwarnings("ignore") # Ortiqcha warning larni berkitadi


In [15]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
columns=[
    "age",
    "height(cm)",
    "weight(kg)",
    "waist(cm)",
    "eyesight(left)",
    "eyesight(right)",
    # "hearing(left)",
    # "hearing(right)",
    "systolic",
    "relaxation",
    "fasting blood sugar",
    "Cholesterol",
    "triglyceride",
    "HDL",
    "LDL",
    "hemoglobin",
    # "Urine protein",
    "serum creatinine",
    "AST",
    "ALT",
    "Gtp",
    "dental caries"
]
X = train[columns]
y = train['smoking']


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lst=["gini", "entropy", "log_loss"]
roc=[]

for crit in lst:
    model =RandomForestClassifier(max_depth=10, max_features=6, min_samples_split=4, max_samples=0.791, criterion=crit)
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    roc.append(roc_auc)
for i in range(3):
    if max(roc)==roc[i]:
        Lit=lst[i]
print("Train-test split orqali ROC AUC:", max(roc), Lit)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
model =RandomForestClassifier(max_depth=10, max_features=6, min_samples_split=4, min_samples_leaf=8 ,max_samples=0.791, criterion=Lit, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(roc_auc)
# 0.8858915865087511


Train-test split orqali ROC AUC: 0.8839560802537686 entropy
0.8858915865087511


In [18]:
model.fit(X, y)
y_pred_final = model.predict_proba(test[columns])[:, 1]
sub = pd.read_csv('sample_submission.csv')
sub['smoking'] = y_pred_final
sub.to_csv('Sample.csv', index=False)
sub

,id,smoking
0,15000,0.739045
1,15001,0.011044
2,15002,0.009687
3,15003,0.041360
4,15004,0.465136
...,...,...
9995,24995,0.339758
9996,24996,0.743415
9997,24997,0.269200
9998,24998,0.006809
